In [1]:
# 패키지 로드
import geopandas as gpd
import pandas as pd
import os

In [2]:
# GDAL_DATA 환경 변수 경로 설정
os.environ['GDAL_DATA'] = r'C:\Users\TG\miniconda3\envs\moonsoo1\Library\share\gdal'

In [3]:
# shp 파일 읽기(한글 속성값 때문에 EUC-KR 인코딩 지정)
data_shp = gpd.read_file('D:/GIS/행정구역/리/동리_유역_병합.shp', encoding='euc-kr')

data_shp

,CODE,DONGLI,법정동코드,법정동명,시도,시군구,읍면동,리,AREA_TOTAL,SW_NAME,AREA_WATER,RATIO,geometry
0,5111010100,봉의동,5111010100,강원특별자치도 춘천시 봉의동,강원특별자치도,춘천시,봉의동,None,177444.458,북한C,1.774445e+05,100.0,"POLYGON ((263858.449 587613.572, 263859.62 587..."
1,5111010200,요선동,5111010200,강원특별자치도 춘천시 요선동,강원특별자치도,춘천시,요선동,None,88350.298,북한C,8.835030e+04,100.0,"POLYGON ((263949.193 587360.716, 263974.087 58..."
2,5111010300,낙원동,5111010300,강원특별자치도 춘천시 낙원동,강원특별자치도,춘천시,낙원동,None,107927.810,북한C,1.079278e+05,100.0,"POLYGON ((263564.29 586554.824, 263559.02 5865..."
3,5111010400,중앙로1가,5111010400,강원특별자치도 춘천시 중앙로1가,강원특별자치도,춘천시,중앙로1가,None,37470.002,북한C,3.747000e+04,100.0,"POLYGON ((264184.523 587301.17, 264144.988 587..."
4,5111010500,중앙로2가,5111010500,강원특별자치도 춘천시 중앙로2가,강원특별자치도,춘천시,중앙로2가,None,44367.327,북한C,4.436733e+04,100.0,"POLYGON ((263722.617 586664.076, 263706.017 58..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1924,None,None,None,None,None,None,None,None,NaN,소양A,1.165400e+01,NaN,"MULTIPOLYGON (((325914.33 606777.035, 325910.6..."
1925,None,None,None,None,None,None,None,None,NaN,인북A,1.804567e+08,NaN,"MULTIPOLYGON (((297052.084 655909.91, 297163.9..."
1926,None,None,None,None,None,None,None,None,NaN,소양B,2.679938e+05,NaN,"MULTIPOLYGON (((302726.162 587979.488, 302791...."
1927,None,None,None,None,None,None,None,None,NaN,제천A,3.800844e+08,NaN,"MULTIPOLYGON (((295445.848 523282.229, 295443...."


In [4]:
# 필요 없는 열(geometry) 제거
data_shp = data_shp.drop(columns=['geometry'])

# AREA_TOTAL, AREA_WATER, RATIO 필드의 값이 NAN이거나 0인 행 제거
data_shp = data_shp.dropna(subset=['AREA_TOTAL', 'AREA_WATER', 'RATIO'])
data_shp = data_shp[(data_shp['AREA_TOTAL'] != 0) & (data_shp['AREA_WATER'] != 0) & (data_shp['RATIO'] != 0)]

data_shp

,CODE,DONGLI,법정동코드,법정동명,시도,시군구,읍면동,리,AREA_TOTAL,SW_NAME,AREA_WATER,RATIO
0,5111010100,봉의동,5111010100,강원특별자치도 춘천시 봉의동,강원특별자치도,춘천시,봉의동,None,1.774445e+05,북한C,1.774445e+05,100.0
1,5111010200,요선동,5111010200,강원특별자치도 춘천시 요선동,강원특별자치도,춘천시,요선동,None,8.835030e+04,북한C,8.835030e+04,100.0
2,5111010300,낙원동,5111010300,강원특별자치도 춘천시 낙원동,강원특별자치도,춘천시,낙원동,None,1.079278e+05,북한C,1.079278e+05,100.0
3,5111010400,중앙로1가,5111010400,강원특별자치도 춘천시 중앙로1가,강원특별자치도,춘천시,중앙로1가,None,3.747000e+04,북한C,3.747000e+04,100.0
4,5111010500,중앙로2가,5111010500,강원특별자치도 춘천시 중앙로2가,강원특별자치도,춘천시,중앙로2가,None,4.436733e+04,북한C,4.436733e+04,100.0
...,...,...,...,...,...,...,...,...,...,...,...,...
1902,5183035038,장산리,5183035038,강원특별자치도 양양군 강현면 장산리,강원특별자치도,양양군,강현면,장산리,2.518901e+06,None,2.518901e+06,100.0
1903,5183035039,전진리,5183035039,강원특별자치도 양양군 강현면 전진리,강원특별자치도,양양군,강현면,전진리,9.178029e+05,None,9.178029e+05,100.0
1904,5183035040,주청리,5183035040,강원특별자치도 양양군 강현면 주청리,강원특별자치도,양양군,강현면,주청리,4.658669e+05,None,4.658669e+05,100.0
1905,5183035041,적은리,5183035041,강원특별자치도 양양군 강현면 적은리,강원특별자치도,양양군,강현면,적은리,1.361706e+06,None,1.361706e+06,100.0


In [5]:
# AREA_TOTAL에서 AREA_WATER의 비중 계산(백분율 소수점 둘째자리까지)
data_shp['ratio'] = (data_shp['AREA_WATER'] / data_shp['AREA_TOTAL']) * 100
data_shp['ratio'] = data_shp['ratio'].round(2)

# RATIO_mod 열을 추가하고, RATIO 열의 값이 5미만인 경우 0으로 설정, 95 이상인 경우 100으로 설정, 이외의 경우 RATIO의 값을 그대로 사용
data_shp['RATIO_mod'] = data_shp['ratio'].apply(lambda x: 0 if x < 5 else (100 if x >= 95 else x))

# count 열을 추가하고, "DONGLI"와 "시군구", "읍면동"열의 값이 동일한 행의 갯수를 세어서 표시
data_shp['count'] = data_shp.groupby(['DONGLI', '시군구', '읍면동'])['DONGLI'].transform('count')

data_shp

,CODE,DONGLI,법정동코드,법정동명,시도,시군구,읍면동,리,AREA_TOTAL,SW_NAME,AREA_WATER,RATIO,ratio,RATIO_mod,count
0,5111010100,봉의동,5111010100,강원특별자치도 춘천시 봉의동,강원특별자치도,춘천시,봉의동,None,1.774445e+05,북한C,1.774445e+05,100.0,100.0,100.0,1
1,5111010200,요선동,5111010200,강원특별자치도 춘천시 요선동,강원특별자치도,춘천시,요선동,None,8.835030e+04,북한C,8.835030e+04,100.0,100.0,100.0,1
2,5111010300,낙원동,5111010300,강원특별자치도 춘천시 낙원동,강원특별자치도,춘천시,낙원동,None,1.079278e+05,북한C,1.079278e+05,100.0,100.0,100.0,1
3,5111010400,중앙로1가,5111010400,강원특별자치도 춘천시 중앙로1가,강원특별자치도,춘천시,중앙로1가,None,3.747000e+04,북한C,3.747000e+04,100.0,100.0,100.0,1
4,5111010500,중앙로2가,5111010500,강원특별자치도 춘천시 중앙로2가,강원특별자치도,춘천시,중앙로2가,None,4.436733e+04,북한C,4.436733e+04,100.0,100.0,100.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1902,5183035038,장산리,5183035038,강원특별자치도 양양군 강현면 장산리,강원특별자치도,양양군,강현면,장산리,2.518901e+06,None,2.518901e+06,100.0,100.0,100.0,1
1903,5183035039,전진리,5183035039,강원특별자치도 양양군 강현면 전진리,강원특별자치도,양양군,강현면,전진리,9.178029e+05,None,9.178029e+05,100.0,100.0,100.0,1
1904,5183035040,주청리,5183035040,강원특별자치도 양양군 강현면 주청리,강원특별자치도,양양군,강현면,주청리,4.658669e+05,None,4.658669e+05,100.0,100.0,100.0,1
1905,5183035041,적은리,5183035041,강원특별자치도 양양군 강현면 적은리,강원특별자치도,양양군,강현면,적은리,1.361706e+06,None,1.361706e+06,100.0,100.0,100.0,1
